# Plan & Execute
Inspiration: https://blog.langchain.dev/planning-agents/

Plan & Execute Agents consists of two basic components:

1. A **planner**, which prompts an (usually large and powerful) LLM to generate a multi-step plan to complete a large task.
2. An **executor**, (potentially many) which accept the user query and a step in the plan and invoke 1 or more tools to complete that task.

In [ ]:
import os

# Set up API keys
os.environ["OPENAI_API_KEY"] = ""


## Setting up a basic ReAct Agent as the Executor

In [3]:
from langchain_community.tools import DuckDuckGoSearchRun  # Initialize the tool
search_ddg_tool = DuckDuckGoSearchRun()

tools = [search_ddg_tool]

In [4]:
from langchain import hub
print(dir(hub))

['Any', 'BasePromptTemplate', 'Optional', 'Sequence', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_get_client', 'annotations', 'dumps', 'json', 'loads', 'pull', 'push']


In [ ]:

from langchain_openai import ChatOpenAI

from langgraph.prebuilt import create_react_agent

# Get the prompt to use - you can modify this!
# prompt = hub.pull("wfh/react-agent-executor")
prompt = hub.pull("hwchase17/react")
prompt.pretty_print()

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


langchain_core.prompts.prompt.PromptTemplate

In [12]:

from langchain.agents import AgentExecutor, create_react_agent
# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-4o-mini")
agent = create_react_agent(llm, tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [13]:
response = agent_executor.invoke({"input": "user: Who won the most recent Ravens game?"})

In [17]:
response['output']

'The Baltimore Ravens lost their most recent game against the Houston Texans, 44-10.'

## The Planner

In [18]:
from pydantic import BaseModel, Field
from typing import List
from langchain_core.prompts import ChatPromptTemplate

class Plan(BaseModel):
    """Plan to follow in future"""

    steps: List[str] = Field(
        description="different steps to follow, should be in sorted order"
    )

planner_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """For the given objective, come up with a simple step by step plan. \
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.""",
        ),
        ("placeholder", "{messages}"),
    ]
)
planner = planner_prompt | ChatOpenAI(
    model="gpt-4o-mini", temperature=0
).with_structured_output(Plan)


In [19]:
plan = planner.invoke(
    {
        "messages": [
            ("user", "what is the hometown of the QB of the winner of the most recent Ravens game?")
        ]
    }
)
plan

Plan(steps=['Identify the most recent game played by the Baltimore Ravens.', 'Determine the outcome of that game (who won).', 'Identify the starting quarterback (QB) of the winning team.', 'Research the hometown of that quarterback.'])

In [20]:
from typing import Union


class Response(BaseModel):
    """Response to user."""

    response: str


class Act(BaseModel):
    """Action to perform."""

    action: Union[Response, Plan] = Field(
        description="Action to perform. If you want to respond to user, use Response. "
        "If you need to further use tools to get the answer, use Plan."
    )


replanner_prompt = ChatPromptTemplate.from_template(
    """For the given objective, come up with a simple step by step plan. \
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.

Your objective was this:
{input}

Your original plan was this:
{plan}

You have currently done the following steps:
{past_steps}

Update your plan accordingly. If no more steps are needed and you can return to the user, then respond with that. Otherwise, fill out the plan. Only add steps to the plan that still NEED to be done. Do not return previously done steps as part of the plan."""
)


replanner = replanner_prompt | ChatOpenAI(
    model="gpt-4o", temperature=0
).with_structured_output(Act)

In [21]:
act = replanner.invoke(
    {
        "input": "what is the hometown of the QB of the winner of the most recent Ravens game?",
        "plan": plan,
        "past_steps": [('Identify the most recent game played by the Baltimore Ravens.', "They played the Steelers")],
        
    }
)

In [22]:
# New steps
new_plan = act.action


In [23]:
type(new_plan)

__main__.Plan

In [24]:
new_plan.steps

['Determine the outcome of the game between the Baltimore Ravens and the Pittsburgh Steelers (who won).',
 'Identify the starting quarterback (QB) of the winning team.',
 'Research the hometown of that quarterback.']

## Building the Graph

In [25]:
import operator
from typing import Annotated, List, Tuple
from typing_extensions import TypedDict

class PlanExecute(TypedDict):  # our graph state (short term memory in between the steps)
    input: str
    plan: List[str]
    past_steps: Annotated[List[Tuple], operator.add]
    response: str

In [ ]:
from typing import Literal
from langgraph.graph import END


def execute_step(state: PlanExecute):
    plan = state["plan"]
    plan_str = "\n".join(f"{i+1}. {step}" for i, step in enumerate(plan))
    task = plan[0]
    task_formatted = f"""For the following plan:
{plan_str}\n\nYou are tasked with executing step {1}, {task}."""
    print('-----------------------')
    print('Formatted Task at Execute', task_formatted)
    print('-----------------------')
    agent_response = agent_executor.invoke(
        {"messages": [("user", task_formatted)]}
    )
    
    return {
        "past_steps": [(task, agent_response["messages"][-1].content)],
    }

def plan_step(state: PlanExecute):
    plan = planner.invoke({"messages": [("user", state["input"])]})
    return {"plan": plan.steps}


def replan_step(state: PlanExecute):
    output = replanner.invoke(state)
    if isinstance(output.action, Response):
        print('A response was given!')
        return {"response": output.action.response}
    else:
        return {"plan": output.action.steps}


def should_end(state: PlanExecute):
    if "response" in state and state["response"]:
        return END
    else:
        return "executor"

In [ ]:
from langgraph.graph import StateGraph, START

workflow = StateGraph(PlanExecute)

# Add the plan node
workflow.add_node("planner", plan_step)

# Add the execution step
workflow.add_node("executor", execute_step)

# Add a replan node
workflow.add_node("replan", replan_step)

workflow.add_edge(START, "planner")

# From plan we go to agent
workflow.add_edge("planner", "executor")

# From agent, we replan
workflow.add_edge("executor", "replan")

workflow.add_conditional_edges(
    "replan",
    # Next, we pass in the function that will determine which node is called next.
    should_end,
    ["executor", END],
)

In [ ]:
# Compile the graph
app = workflow.compile()
from IPython.display import Image, display

display(Image(app.get_graph(xray=True).draw_mermaid_png()))

## Using our Plan & Execute Agent

In [ ]:
inputs = {"input": "what is the hometown of the QB of the winner of the Ravens game on 11/17/2024"}

for event in app.stream(inputs):
    for node_name, output in event.items():
        print(f'Node: {node_name}. Output: {output}')

In [ ]:
for event in app.stream(inputs, stream_mode=['values']):  # Listen for state updates
    print(len(event[1].get('past_steps')), event[1].get('past_steps'))

In [ ]:
final_state = event[1]
final_state